In [ ]:
# default_exp dwpc

# DWPC

> DWPC calculation

In [ ]:
#hide
from nbdev.showdoc import *
from neo4jtools import SimpleNeo4jHTTPAPIClient

In [ ]:
#export
def make_dwpc_query(genes, reltype, hops, only_relations_with_pmid=False):
    '''
    Make a cypher query to calculate DWPC 
    
    input parameters
    
    genes : gene list
    reltype : edge type
    hops : number of hops to calculate
    only_relations_with_pmid : include only relations with pubmed_id
    
    output
    query:string
    '''
    qry_template="""MATCH p=(n0:Gene)-[:{reltype}*..{hops}]->(n1:Gene) 
    WHERE n0.name in {genestr}
      AND n1.name in {genestr}
      AND n0 <> n1
      {conditionstr}
    WITH 
        n0.name as source_name
      , n1.name as target_name
      , p as path
      , [r in relationships(p)|r.dwpc_score] as dwpc_scores
      , [r in relationships(p) | r.pubmed_id] as pubmed_ids
    WITH 
        source_name
      , target_name
      , length(path) as len
      , reduce(acc=1, d in dwpc_scores|acc*d) as dwpc_score
      , [n IN nodes(path) | labels(n)[0]] as node_labels
      , [n IN nodes(path) | n.identifier] as node_identifiers
      , [r IN relationships(path) | type(r)] as rel_types
      , reduce(acc='', s in pubmed_ids | acc+';'+s) as pubmed_ids
    WITH 
        source_name
      , target_name
      , dwpc_score as dwpc
      , [r IN rel_types | split(r, '_')[1]] as rtypes
      , "(:"+node_labels[0]+"{{identifier:'"+ node_identifiers[0] +"'}})" as head
      , [i IN range(0,len-1)| "-[:"+ rel_types[i] +"]->(:"+node_labels[i+1]+"{{identifier:'"+ node_identifiers[i+1] +"'}})" ] as tail
      , substring(pubmed_ids,1) as pubmed_ids
    RETURN 
        source_name
      , target_name
      , dwpc
      , head + reduce(acc='', s in tail |acc+s) as pattern
      , reduce(acc='', s IN rtypes | acc + s) as type_pattern
      , pubmed_ids
    """
    only_if_pubmed_id_exists="AND all(r IN relationships(p) WHERE exists(r.pubmed_id)) "

    genelist_str="{}{}{}".format("['", "','".join(genes), "']")
    
    conditionstr=''
    if only_relations_with_pmid:
        conditionstr=conditionstr + only_if_pubmed_id_exists

    qry=qry_template.format(genestr=genelist_str, 
                            reltype=reltype, 
                            hops=hops, 
                            conditionstr=conditionstr)
    return qry


def patterns_to_query(patterns):
    '''
    Convert pattern list to cypher query
    
    Example
    
    input patterns:list:
    
    ["(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})",
    "(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2533'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})",
    "(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'7827'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})",
    "(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'55243'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})"]
    
    output query:string
    "MATCH p0=(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})  
    MATCH p1=(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2533'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})  
    MATCH p2=(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'7827'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})  
    MATCH p3=(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'55243'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'}) 
    RETURN p0,p1,p2,p3"
    '''
    match_phrases=[]
    for i, pattern in enumerate(patterns):
        match_phrases.append('MATCH p{i}={pattern} '.format(i=i, pattern=pattern))
    
    return_phrase='RETURN ' + ','.join(["p{}".format(i) for i in range(len(patterns))])
    
    query= ' '.join(match_phrases) + return_phrase
    
    return query

# Tests

## Test `make_dwpc_query` 

In [ ]:
genes=['LEF1','TCF7','SERINC5']
reltype='INTERACTS_GiG3'
hops=2

In [ ]:
query=make_dwpc_query(genes, reltype, hops)
query

'MATCH p=(n0:Gene)-[:INTERACTS_GiG3*..2]->(n1:Gene) \n    WHERE n0.name in [\'LEF1\',\'TCF7\',\'SERINC5\']\n      AND n1.name in [\'LEF1\',\'TCF7\',\'SERINC5\']\n      AND n0 <> n1\n      \n    WITH \n        n0.name as source_name\n      , n1.name as target_name\n      , p as path\n      , [r in relationships(p)|r.dwpc_score] as dwpc_scores\n      , [r in relationships(p) | r.pubmed_id] as pubmed_ids\n    WITH \n        source_name\n      , target_name\n      , length(path) as len\n      , reduce(acc=1, d in dwpc_scores|acc*d) as dwpc_score\n      , [n IN nodes(path) | labels(n)[0]] as node_labels\n      , [n IN nodes(path) | n.identifier] as node_identifiers\n      , [r IN relationships(path) | type(r)] as rel_types\n      , reduce(acc=\'\', s in pubmed_ids | acc+\';\'+s) as pubmed_ids\n    WITH \n        source_name\n      , target_name\n      , dwpc_score as dwpc\n      , [r IN rel_types | split(r, \'_\')[1]] as rtypes\n      , "(:"+node_labels[0]+"{identifier:\'"+ node_identifiers

In [ ]:
client=SimpleNeo4jHTTPAPIClient('http://localhost:7474')
client

{"classname": "SimpleNeo4jHTTPAPIClient", "url": "http://localhost:7474", "neo4j_version": "4.4.3", "neo4j_edition": "community", "db": "neo4j", "auth": false}

In [ ]:
result=client.execute_read_query(query)

In [ ]:
assert result['results'][0]['data'], 'The DWPC query is not working.'

## Test `patterns_to_query`

In [ ]:
patterns=["(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})",
    "(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2533'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})",
    "(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'7827'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})",
    "(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'55243'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})"]
patterns

["(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})",
 "(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2533'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})",
 "(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'7827'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})",
 "(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'55243'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})"]

In [ ]:
query=patterns_to_query(patterns)
query

"MATCH p0=(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})  MATCH p1=(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2533'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})  MATCH p2=(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'7827'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'})  MATCH p3=(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'55243'})-[:INTERACTS_GiG3]->(:Gene{identifier:'2534'})-[:INTERACTS_GiG3]->(:Gene{identifier:'201633'}) RETURN p0,p1,p2,p3"

In [ ]:
assert client.execute_read_query(query)['results'][0]['data'], 'Converted query from patterns is not working.'